In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg
from matplotlib import colors

In [ ]:
margin = 0.5

vector_params = {
    'color': ['b', 'r'],
    'width': 0.013,
    'angles': 'xy',
    'scale_units': 'xy',
    'scale': 1
}

space_names = [
    "original",
    "transformed"
]

In [ ]:
def get_unit_circle(n=100):
    xi1 = np.linspace(-1.0, 1.0, n)
    xi2 = np.linspace(1.0, -1.0, n)
    yi1 = np.sqrt(1 - xi1**2)
    yi2 = -np.sqrt(1 - xi2**2)

    xi = np.concatenate((xi1, xi2),axis=0)
    yi = np.concatenate((yi1, yi2),axis=0)
    return np.vstack((xi, yi))

In [ ]:
def plot_2d_vectors(vs, f_vs, shapes=None):
    origin = np.array([[0, 0]])
    vss = np.vstack([origin, vs, f_vs, *shapes])
    fig, axs = plt.subplots(1, 2, figsize=(10,15))
    plt.subplots_adjust(wspace=0.4)

    if shapes:
        axs[0].plot(shapes[0][:,0], shapes[0][:,1])
        axs[1].plot(shapes[1][:,0], shapes[1][:,1])

    for v, f_v, i in zip(vs, f_vs, range(len(vs))):
        axs[0].quiver(*origin[0], *v, **vector_params)
        axs[1].quiver(*origin[0], *f_v, **vector_params)
        axs[0].text(*v, "$\mathbf{x_i}$".replace('i', str(i)), fontsize=11)
        axs[1].text(*f_v, "$\mathbf{f(x_i)}$".replace('i', str(i)), fontsize=11)

    for ax, space_name in zip(axs, space_names):
        ax.set_xlabel('x', fontsize=14)
        ax.set_ylabel('y', fontsize=14)
        ax.set_xlim(
            np.min(vss[:,0]) - margin, np.max(vss[:,0]) + margin)
        ax.set_ylim(
            np.min(vss[:,1]) - margin, np.max(vss[:,1]) + margin)
        ax.set_aspect('equal')
        ax.grid(True)
        ax.set_axisbelow(True)
        ax.set_title(space_name)
        ax.axhline(y=0, color='k')
        ax.axvline(x=0, color='k')

In [ ]:
A = np.array([[1., 2.],
              [0., 1.],
              [1., 1.],
              [0., 0.]])

In [ ]:
x = np.array([[1, 1],
              [2, 2]])

In [ ]:
A @ x

In [ ]:
xs = np.array([[1, 0], [0, 1], [1, 1]/np.sqrt(2)])
A = np.array([[1, -1], [1, 1]])/np.sqrt(2)
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

In [ ]:
xs = np.array([[1, 0], [0, 1], [0, -1], [-1, 0]])
A = np.array([[1, 2], [3, 1]])
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()